In [1]:
#load libraries
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 10)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.0     ✔ rsample      1.0.0
✔ dials        1.0.0     ✔ tune         1.0.0
✔ infer        1.0.2     ✔ workflows    1.0.0
✔ modeldata    1.0.0     ✔ workflowsets 1.0.0
✔ parsnip      1.0.0     ✔ yardstick    1.0.0
✔ recipes      1.0.1     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks r

In [8]:
#Add columns
pulsar_data <- read_csv("pulsar_data.csv", 
                        col_names = c("mean_integrated_profile", 
                                      "stand_dev_integrated_profile", 
                                      "exc_kurtosis_integrated_profile", 
                                      "skew_integrated_profile",
                                      "mean_dmsnr", 
                                      "stand_dev_dmsnr", 
                                      "exc_kurtosis_dmsnr", 
                                      "skew_dmsnr", "class")) 

#Mutate Class as factor variable
pulsar_data <- pulsar_data |>
    mutate(class = as_factor(class))
#pulsar_data 

#Scale data
pulsar_data_scaled <- pulsar_data |> 
    mutate(scaled_skew_integrated_profile = scale(skew_integrated_profile, center = TRUE), 
           scaled_skew_dmsnr = scale(skew_dmsnr, center = TRUE)) |>
    select(scaled_skew_integrated_profile, scaled_skew_dmsnr)
pulsar_data_scaled

Rows: 17898 Columns: 9
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (9): mean_integrated_profile, stand_dev_integrated_profile, exc_kurtosis...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


scaled_skew_integrated_profile,scaled_skew_dmsnr
"<dbl[,1]>","<dbl[,1]>"
-0.40044782,-0.28743009
-0.37052514,0.21157554
-0.11658960,-0.39136253
-0.39016719,-0.48128686
-0.10486323,1.38675525
-0.21945151,0.24913308
-0.30519991,3.51969656
-0.22385830,0.87667607
-0.25939593,0.02924293
